In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import FinalBorealis as dg
import nni

In [ ]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)
    def forward(self, x):
        return self.linear(x)

In [ ]:
def train(model, opt, loss_fn, train_ld, writer, n_epoch=3, textFile='test.txt'):
    ##Iterate through a number of epochs
    for epoch in range(n_epoch):
        ##Training with batches of data
        running_loss = 0.0
        num_iters = len(train_ld)
        for i, data in enumerate(train_ld,0):
            inputs, labels = data[:,:16], data[:,16:]
            #First step: Generate predictions
           
            pred = model(inputs)
           
            #2nd step: Calculate loss
            loss = loss_fn(pred, labels)
            loss.backward()
            opt.step()

            opt.zero_grad()
            running_loss += loss.item()
            if i % 100== 0:
                nni.report_intermediate_result(running_loss)
                running_loss = 0.0

In [ ]:
def test(testLoader, model, writer, textFile):
    #model = linearRegression(16,1)
    #model.load_state_dict(torch.load(load_pretrained_weights))

    offset = 0
    total = 0
    with torch.no_grad():
        for data in testLoader:
            #print(data)
            outputs = model(data[:,:16])
            #print(f'{outputs.shape}')
            for i in range(outputs.shape[0]):
                if i % 4000 == 3999:
                offset += (outputs[i].item() * 100) / data[i, 16:].item()
            total += outputs.shape[0]

    result = offset / total
    nni.report_final_result(result)

In [ ]:
def main(args):
    ## Load the data
    data1 = pd.read_json('./data1.json')
    data2 = pd.read_json('./data2.json')
    data3 = pd.read_json('./data3.json')
    data1 = torch.from_numpy(data1)
    data2 = torch.from_numpy(data2)
    data3 = torch.from_numpy(data3)
    data = torch.cat((data1, torch.cat((data2, data3), 0), 0)
    data = data.float()

    ## Divide the data
    testSize = int(0.1 * len(data))
    trainSize = len(data)-testSize

    trainSet, testSet = torch.utils.data.random_split(data, [trainSize, testSize])

    ## Load the data into tensor
    trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=args['batch_size'], num_workers=0)
    testLoader = torch.utils.data.DataLoader(testSet, batch_size=args['batch_size'], num_workers=0)

    ## Model and parameters
    model = linearRegression(16,1)
    loss_fn = nn.MSELoss()
    opt = torch.optim.SGD(model.parameters(), lr=args['lr'])

    train(model, opt, loss_fn, trainLoader, writer, n_epoch=args['epochs'], textFile=textFile)
    test(testLoader, model, writer, textFile)

In [ ]:
if __name__ == '__main__':
    try:
        params = nni.get_next_parameter()